<a href="https://colab.research.google.com/github/mhuggi/nlp/blob/master/Salmi_Emil_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic NLP exercises

* During these exercises, you will learn basic Python skills required in NLP, for example
  * Reading and processing language data
  * Segmenting text
  * Calculating word frequencies and idf weights

* Exercises are based on tweets downloaded using Twitter API. Both Finnish and English tweets are available, you are free to choose which language you want to work with.


> Finnish: http://dl.turkunlp.org/intro-to-nlp/finnish-tweets-sample.jsonl.gz

> English: http://dl.turkunlp.org/intro-to-nlp/english-tweets-sample.jsonl.gz


* Both files include 10,000 tweets. If processing the whole file takes too much time, you can also read just a subset of the data, for example only 1,000 tweets.


## 1) Read tweets in Python

* Download the file, and read the data in Python
* **The outcome of this exercise** should be a list of tweets, where each tweet is a dictionary including different (key, value) pairs

In [ ]:
!wget -nc http://dl.turkunlp.org/intro-to-nlp/finnish-tweets-sample.jsonl.gz

import gzip
import json
f = gzip.open("finnish-tweets-sample.jsonl.gz", "rt", encoding="utf-8")

lines = f.readlines()

tweets = []
for line in lines:
    d = json.loads(line)
    tweets.append(d)

print(len(lines))
print(type(lines[0]))
print(lines[0])

print(len(tweets))
print(type(tweets[0]['retweeted_status']['text']))
print(tweets[0]['retweeted_status']['text'])

File ‘finnish-tweets-sample.jsonl.gz’ already there; not retrieving.

10000
<class 'str'>
{"retweeted_status": {"retweet_count": 1, "is_quote_status": false, "geo": null, "favorited": false, "in_reply_to_user_id": null, "place": null, "id": 777860632850919424, "possibly_sensitive": false, "coordinates": null, "truncated": false, "id_str": "777860632850919424", "in_reply_to_status_id": null, "source": "<a href=\"http://twitter.com\" rel=\"nofollow\">Twitter Web Client</a>", "in_reply_to_user_id_str": null, "text": "Ranskassa lakiehdotusessa mukana OA ja avoin data, mm oikeudesta rinnakkaisjulkaisemiseen kustantajasta riippumatta  https://t.co/HMwgIDWkeJ", "in_reply_to_screen_name": null, "contributors": null, "retweeted": false, "lang": "fi", "created_at": "Mon Sep 19 13:23:21 +0000 2016", "filter_level": "low", "in_reply_to_status_id_str": null, "favorite_count": 2, "entities": {"symbols": [], "urls": [{"display_url": "openaire.eu/france-final-t\u2026", "url": "https://t.co/HMwgIDWkeJ"

## 2) Extract texts from the tweet jsons

* During these exercises we need only the actual tweet text. Inspect the dictionary and extract the actual text field for each tweet.
* When carefully inspecting the dictionary keys and values, you may see the old Twitter character limit causing unexpect behavior for text. In these cases, are you able to extract the full text?
* **The outcome of this exercise** should be a list of tweets, where each tweet is a string.

In [ ]:
tweet_strings = []
for t in tweets:
  if 'retweeted_status' in t:
    if t['retweeted_status']['truncated']:
      tweet_strings.append(t['retweeted_status']['extended_tweet']['full_text'])
    else:
      tweet_strings.append(t['retweeted_status']['text'])
  else:
    if t['truncated']:
      tweet_strings.append(t['extended_tweet']['full_text'])
    else:
      tweet_strings.append(t['text'])


#tweet_strings =  [tweet['text'] for tweet in tweets]

print(tweet_strings[0])
print(len(tweet_strings))

Ranskassa lakiehdotusessa mukana OA ja avoin data, mm oikeudesta rinnakkaisjulkaisemiseen kustantajasta riippumatta  https://t.co/HMwgIDWkeJ
10000


## 3) Segment tweets

* Segment tweets using the UDPipe machine learned model, remember to select the correct language.

> English model: https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/en.segmenter.udpipe

> Finnish model: https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/fi.segmenter.udpipe

* Note that the segmentation model was not trained on tweets, so it may have difficulties in some cases. Inspect the output to get an idea how well it performs on tweets.
* Note: In case of the notebook cell dies while trying to load/run the model, the most typical reason is wrong file path or name, or incorrectly downloaded model.
* **The output of this excercise** should be a list of segmented tweets, where each tweet is a string.

In [ ]:
!wget -nc https://github.com/TurkuNLP/intro-to-nlp/raw/master/Data/fi.segmenter.udpipe

!pip3 install ufal.udpipe

import ufal.udpipe as udpipe

model = udpipe.Model.load("fi.segmenter.udpipe")
pipeline = udpipe.Pipeline(model, "tokenize", "none", "none", "horizontal")

segmented_tweets = []
for t in tweet_strings:
  segmented_tweets.append(pipeline.process(t))

print(segmented_tweets)



File ‘fi.segmenter.udpipe’ already there; not retrieving.

['Ranskassa lakiehdotusessa mukana OA ja avoin data , mm oikeudesta rinnakkaisjulkaisemiseen kustantajasta riippumatta https://t.co/HMwgIDWkeJ\n', 'Miten toteutuu tasa-arvo kuuroilla Eritreassa # viittomakieli # lähetys https://t.co/gPmPRMqHsr https://t.co/0c5rUnRvc0\n', '@lauriojain No toisaalta meillä oli hallitus joka uumoili Suur-Suomesta , luovutti juutalaisia natseille ja soti niiden kanssa .\n', '@Muotioikeus ensimmäinen tilaisuus 27.9 klo 17- Krogeruksella w/ vaateliike # vestiarium 👍 save the date , lisäinfoa seuraa vielä TÄNÄÄN !\n', '@filosofinenkok @winninghelix Elviksestä en ole varma , mutta Isaac Asimov elää .\n# TeknologianMoraali\n', '855519 kahdeksansataaviisikymmentäviisituhattaviisisataayhdeksäntoista\n', '@terolahtinen Saksalla menee erinomaisen hyvin , syistä siihen olemme ilmeisesti eri mieltä .\n@nollapiste @molkko\n', '@HkiCityTrail lähestyy👟 15.10. juostaan poluilla , hypitään kallioilla ja nautitaan l

## 4) Calculate word frequencies

* Calculate a word frequency list (how many times each word appears) based on the tweets. Which are the most common words appearing in the data?
* Calculate the size of your vocabulary (how many unique words there are).
* **The output of this excercise** should be a sorted list of X most common words and their frequencies, and the number of unique words in the data.

In [ ]:
from collections import Counter

token_counter = Counter()
for t in segmented_tweets:
  tokenized = pipeline.process(t)
  tokens = tokenized.split()
  token_counter.update(tokens)

print("Most common tokens:", token_counter.most_common(20))
print("Vocalubary size:", len(token_counter))

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
filtered_tokens = []
punctuation_chars = '. , : ( ) ! ? " = & - ; ... \\ # - / – ” |'.split()

for word, count in token_counter.most_common(150):
  if word.lower() in stopwords.words("finnish") or word in punctuation_chars:
    continue
  filtered_tokens.append((word, count))

print("Filtered tokens: ", filtered_tokens[:20])
print("Filtered count: ", len(token_counter))

Most common tokens: [('.', 8899), ('#', 6813), (',', 5811), ('ja', 3621), ('on', 2999), ('!', 2284), ('?', 1537), (':', 1426), ('"', 1315), ('ei', 1212), ('että', 958), ('-', 647), ('”', 628), (')', 616), ('(', 604), ('se', 583), ('kun', 470), ('ole', 453), ('mutta', 444), ('...', 440)]
Vocalubary size: 63345
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Filtered tokens:  [('myös', 398), ('voi', 337), ('jo', 287), ('euroviisut', 272), ('tänään', 228), ('Suomen', 221), ('ihan', 219), ('vielä', 210), ('saa', 197), ('vain', 190), ('sitten', 184), ('tulee', 183), ('kaikki', 176), ('hyvä', 175), ('kyllä', 166), ('paljon', 154), ('aina', 145), ('pitää', 133), ('Suomi', 133), ('koko', 132)]
Filtered count:  63345


## 5) Calculate idf weights

* Calculate idf weight for each word appearing in the data (one tweet = one document), and print top 20 words with lowest and highest idf values.
* Can you think of a reason why someone could claim that tf does not have a high impact when processing tweets?
* **The output of this excercise** should be a list of words sorted by their idf weights.


In [ ]:
print(token_counter)

Counter({'.': 8899, '#': 6813, ',': 5811, 'ja': 3621, 'on': 2999, '!': 2284, '?': 1537, ':': 1426, '"': 1315, 'ei': 1212, 'että': 958, '-': 647, '”': 628, ')': 616, '(': 604, 'se': 583, 'kun': 470, 'ole': 453, 'mutta': 444, '...': 440, '–': 428, 'oli': 424, 'niin': 424, 'myös': 398, 'nyt': 383, 'voi': 337, 'kuin': 332, 'olla': 305, 'jos': 303, 'jo': 287, 'tai': 280, 'Ei': 278, 'ovat': 277, 'euroviisut': 272, 'sen': 248, 'vaan': 245, 'sitä': 237, 'joka': 232, 'tänään': 228, 'Suomen': 221, 'ihan': 219, 'kanssa': 210, 'vielä': 210, 'saa': 197, 'vain': 190, 'en': 189, 'sitten': 184, 'tulee': 183, 'Ja': 182, 'mitä': 181, 'siitä': 178, 'kaikki': 176, 'hyvä': 175, 'olisi': 171, 'Nyt': 170, 'mukaan': 169, 'kyllä': 166, 'En': 155, 'paljon': 154, 'Jos': 153, 'ollut': 148, ';': 146, 'aina': 145, 'et': 145, 'ne': 143, 'tämä': 138, 'vaikka': 137, 'pitää': 133, 'Suomi': 133, 'koko': 132, 'ku': 131, 'lisää': 128, 'miten': 128, 'Se': 127, 'taas': 126, 'jälkeen': 125, 'Kiitos': 124, 'mitään': 122, 'klo

In [ ]:
import math

df = {}

for w in filtered_tokens:
  for d in tweet_strings:
    if w[0] in d:
      if w[0] in df:
        df[w[0]] += 1
      else:
        df[w[0]] = 1
idf = []
for k, v in df.items():
  idf.append((k, len(tweet_strings)/v, v))

sorted_idf = sorted(idf, key=lambda d: d[0])
print(sorted_idf)
tf = []

for k, v in filtered_tokens:
  tf.append((k, v/len(tweet_strings)))
sorted_tf = sorted(tf, key=lambda d: d[0])

tf_idf = []

for i in range(len(sorted_idf)):
  idf_weight = sorted_tf[i][1] * math.log(sorted_idf[i][1])
  tf_idf.append((sorted_tf[i][0], idf_weight, f"(M={len(tweet_strings)}, df={sorted_idf[i][2]})"))

sorted_tf_idf = sorted(tf_idf, key=lambda d: d[1], reverse=True)
print(sorted_tf_idf)

[('#mmkisat', 57.47126436781609, 174), ('%', 112.35955056179775, 89), ("'", 30.67484662576687, 326), ('..', 18.115942028985508, 552), ('@BTS_twt', 129.87012987012986, 77), ('@YouTube', 68.4931506849315, 146), ('Helsingin', 135.13513513513513, 74), ('Kiitos', 80.64516129032258, 124), ('Kyllä', 112.35955056179775, 89), ('Leijonat', 94.33962264150944, 106), ('Lue', 125.0, 80), ('No', 37.878787878787875, 264), ('Onko', 98.03921568627452, 102), ('Suomen', 40.0, 250), ('Suomessa', 91.74311926605505, 109), ('Suomi', 46.51162790697674, 215), ('Tänään', 100.0, 100), ('aika', 17.123287671232877, 584), ('aina', 25.0, 400), ('alkaa', 111.11111111111111, 90), ('edes', 66.2251655629139, 151), ('eikä', 104.16666666666667, 96), ('enemmän', 109.89010989010988, 91), ('ennen', 111.11111111111111, 90), ('enää', 103.09278350515464, 97), ('euroviisut', 36.496350364963504, 274), ('hyvin', 58.13953488372093, 172), ('hyvä', 26.455026455026456, 378), ('ihan', 32.78688524590164, 305), ('ilman', 55.55555555555556

## 6) Duplicates or near duplicates

* Check whether we have duplicate tweets (in terms of text field only) in our dataset. Duplicate tweet means here that the exactly same tweet text appears more than once in our dataset.
* Note: It makes sense to check the duplicates using original tweet texts as the texts were before segmentation. I would also recommend using the full 10,000 dataset here in order to get higher chance of seeing duplicates (this does not require heavy computing).
* Try to check whether tweets have additional near-duplicates. Near duplicate means here that tweet text is almost the same in two or more tweets. Ponder what kind of near duplicates there could be and how to find those. Start by considering for example different normalization techniques. Implement some of the techniques you considered.
* **The outcome of this exercise** should be a number of unique tweets in our dataset (with possibly counting also which are the most common duplicates) as well as the number of unique tweets after removing also near duplicates.

In [ ]:
no_dupl = list(dict.fromkeys(tweet_strings))
print("No duplicates: ", len(no_dupl))

# check for duplicates after segmentation
seg_dupl = list(dict.fromkeys(segmented_tweets))
print("No duplicates after segmentation: ", len(seg_dupl))

# Remove stopwords and punctuation chars
nltk.download('punkt')

from nltk.tokenize import word_tokenize

tokenized_t = []
for t in segmented_tweets:
  tweet = word_tokenize(t)
  tokens_no_sw = [w for w in tweet if not w in stopwords.words("finnish") or w in punctuation_chars]
  no_sw_t = (" ").join(tokens_no_sw)
  tokenized_t.append(no_sw_t)

no_sw_dupl = list(dict.fromkeys(tokenized_t))
print("No stopwords duplicates:", len(no_sw_dupl))

No duplicates:  9773
No duplicates after segmentation:  9760
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No stopwords duplicates: 9758
